# Module I - Climate Regime
<hr>
This module performs climate data analysis and compiling general agro-climatic indicators. These general agro-climatic indicators summarize climatic profiles in the study area for each grid. The key input data for this module is the climatic data, and the geographical and terrain data.
<hr>


## Set Up

First, install any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [1]:
'''import supporting libraries'''
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys

Set up the working directory where the PyAEZ project is located.

In [ ]:
'Set the working directory'
work_dir = r'/Users/ming-mayhu/Desktop/毕业论文/qtp-pyaez/qtp_pyaez' # Please change this to your working directory
os.chdir(work_dir)
os.getcwd()


'/Users/ming-mayhu/Desktop/毕业论文/qtp-pyaez/qtp_pyaez'

Check and create data output folder.

In [4]:
'Create output folder if it does not exist'
import os
folder_path = './output/module1/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")

Folder created successfully.


<hr>

## MODULE 1: Climate Regime

First, initiate the Module 1 Class instance.

In [ ]:
%load_ext autoreload
%autoreload 2
# Import Module 1 and initate Object Class
from pyaez import ClimateRegime
clim_reg = ClimateRegime.ClimateRegime()

# Importing UtilitiesCalc
from pyaez import UtilitiesCalc
obj_util = UtilitiesCalc.UtilitiesCalc()

### Input Data
### Importing the climate dataset and the geographical data/rasters.

The package expects six climate variables, as daily or monthly observations, as Numpy arrays.
Arrays must be 3-dimensional, with the third axes containing the time dimension.
Unit of measures are expected as follows:
- Minimum temperature = Degree Celsius
- Maximum temperature = Degree Celsius
- Precipitation = Accumulated mm / day (or per month)
- Solar radiation = W/m^2
- Wind speed = Average m/s
- Relative humidity = Average fraction (0 to 1)

In addition to climate data, the system requires:
- A binary admin_mask, with 0 and 1 values. 0 pixels values will be not executed, while 1 pixels values will be executed
- An elevation layer
- Soil/terrain/special land cover classes
  

**All the datasets must have the same shape.**

In [ ]:
'''reading climate data'''
# Importing the climate data
max_temp = np.load(r'./input/climate/max_temp.npy')  # maximum temperature
min_temp = np.load(r'./input/climate/min_temp.npy')  # minimum temperature
precipitation = np.load(r'./input/climate/precipitation.npy')  # precipitation
rel_humidity = np.load(r'./input/climate/relative_humidity.npy')  # relative humidity
wind_speed = np.load(r'./input/climate/wind_speed.npy') # wind speed measured at two meters
short_rad = np.load(r'./input/climate/short_rad.npy')  # shortwave radiation

TODO: 
# Load the geographical data/rasters
mask_path=r'../data_input/LAO_Admin.tif'
mask = gdal.Open(r'../data_input/LAO_Admin.tif').ReadAsArray()
elevation = gdal.Open(r'../data_input/LAO_Elevation.tif').ReadAsArray()
soil_terrain_lulc = gdal.Open(r'../data_input/LAO_soil_terrain_lulc.tif').ReadAsArray()

This section contains parameters that can be modified by the user:
- lat_min = minimum latitude of analysis
- lat_max = maximum latitude of analysis
- mask_value = the value in the admin_mask to exclude from the analysis (typically 0)
- daily = whether climate input data are daily (True) or monthly (False)

In [ ]:
# Define the Area-Of-Interest's geographical extents
lat_min = 13.87
lat_max = 22.59
mask_value = 0  # pixel value in admin_mask to exclude from the analysis
daily = False #Type of climate data = True: daily, False: monthly

### Loading the imported data into the Object Class ('*clim_reg*' Class)

In [ ]:
clim_reg.setStudyAreaMask(mask, mask_value)
clim_reg.setLocationTerrainData(lat_min, lat_max, elevation)
if daily:
    clim_reg.setDailyClimateData(
        min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
else:
    clim_reg.setMonthlyClimateData(
        min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

In [ ]:
'Releasing the memory of input climate data -- free up some RAM space'
del(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

NOTE: After loading the data into the *clim_reg* Class, all the parameters will be converted to DAILY DATA and calculated as other parameters. 
These new parameters are available and can be called into use as:
- *clim_reg.minT_daily* (minimum temperature)
- *clim_reg.maxT_daily* (maximum temperature)
- *clim_reg.meanT_daily* (mean temperature)
- *clim_reg.meanT_daily_sealevel* (mean temperature, corrected to sea-level)
- *clim_reg.totalPrec_daily* (total precipitation)
- *clim_reg.pet_daily* (reference evapotranspiration, ETo)
- *clim_reg.P_by_PET_daily* (ratio of precipitation over ETo)
  
